# Notebook 04: MCP Tools - Integrating External Tools with Agents

## 🎯 What is This Notebook About?

Welcome to Notebook 04! This notebook introduces you to **MCP (Model Context Protocol)** - a standardized way to integrate external tools and services with autonomous agents.

**What we'll do:**
- Understand what MCP is and why it matters
- Learn how MCP differs from client-side tools
- Connect agents to external MCP servers (like MongoDB)
- See agents use MCP tools to interact with databases and external services
- Understand when to use MCP vs. client-side tools

**Why this matters:**
- In Notebook 02, we built client-side tools (like Wikipedia search) that run in your Python process
- MCP tools connect agents to **external services** - databases, APIs, monitoring systems, anything!
- This is how you build production agents that interact with your real IT infrastructure

**The big picture:**
Think of client-side tools as "local helpers" and MCP tools as "remote services." MCP provides a standardized protocol so agents can safely connect to external systems - databases, ticketing systems, monitoring APIs, cloud services, and more.

**Real-world impact:**
- **Database integration:** Agents can query databases, update records, analyze data
- **API integration:** Connect to any REST API, GraphQL endpoint, or service
- **Infrastructure tools:** Integrate with monitoring systems, service management, cloud platforms
- **Standardized protocol:** One way to connect to everything, making agents portable and reusable

**The fun part:** You'll see agents querying MongoDB databases and using external tools - just like they would in production!

---

## 📚 Key Concepts Explained

### What is MCP (Model Context Protocol)?

**MCP (Model Context Protocol)** is a standardized protocol for connecting AI agents to external tools and services.

**What it is:** A protocol (like HTTP or REST) that defines how agents communicate with external tools.

**Why it matters:** Instead of every tool having its own integration method, MCP provides one standard way to connect agents to any external service.

**Think of it like:** 
- **Client-side tools** = Local functions in your Python code (like the Wikipedia search from Notebook 02)
- **MCP tools** = Remote services you connect to via a standard protocol (like databases, APIs, monitoring systems)

**How it works:**
1. An **MCP server** exposes tools (like a database query tool, API call tool, etc.)
2. The **agent** connects to the MCP server
3. The agent can discover available tools and call them
4. Tools execute on the server and return results to the agent

### MCP vs. Client-Side Tools

**Client-Side Tools (Notebook 02):**
- ✅ Simple to create (just Python functions)
- ✅ Fast (run in your process)
- ✅ No network overhead
- ❌ Limited to what you can do in Python
- ❌ Can't access remote services easily

**MCP Tools (This Notebook):**
- ✅ Connect to external services (databases, APIs, etc.)
- ✅ Standardized protocol (works with any MCP-compatible service)
- ✅ Can be shared across multiple agents
- ✅ Run on dedicated servers (better for production)
- ❌ Requires MCP server setup
- ❌ Network latency

**Think of it like:**
- **Client-side tools** = Local library books (fast, easy, but limited selection)
- **MCP tools** = Inter-library loan system (access to everything, but requires connection)

**When to use each:**
- **Client-side:** Simple operations, no external dependencies, prototyping
- **MCP:** External services, production systems, shared tools across agents

### MCP Server

An **MCP server** is a service that exposes tools via the MCP protocol.

**What it is:** A server (like a web API) that implements the MCP protocol and provides tools to agents.

**Why it matters:** MCP servers let you expose any service as tools - databases, APIs, monitoring systems, anything!

**Think of it like:** A restaurant menu. The MCP server is the restaurant (provides the menu of tools), and the agent is the customer (orders from the menu).

**Examples:**
- **MongoDB MCP Server:** Exposes database operations as tools (query, insert, update)
- **Terminal MCP Server:** Exposes safe terminal commands as tools
- **Monitoring MCP Server:** Exposes monitoring API calls as tools
- **Custom MCP Server:** You can build your own for any service!

### MCP Tools in Action

When an agent uses an MCP tool:
1. **Agent reasons:** "I need to query the database"
2. **Agent discovers tools:** Checks what MCP tools are available
3. **Agent calls tool:** Sends request to MCP server
4. **MCP server executes:** Runs the actual operation (database query, API call, etc.)
5. **Results return:** MCP server sends results back to agent
6. **Agent continues:** Uses results to complete the task

**Think of it like:** A customer service agent calling a specialist department. The agent doesn't know how to do everything, but knows who to call (which tool to use) and how to ask (MCP protocol).

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- ✅ Understand what MCP is and how it works
- ✅ Know when to use MCP tools vs. client-side tools
- ✅ Connect agents to MCP servers
- ✅ See agents use MCP tools to interact with external services
- ✅ Understand how to build production-ready agents with external integrations

---

## ⚠️ Prerequisites

Before starting this notebook, make sure you have:
- ✅ Completed Notebook 02: Building a Simple Agent (understanding of agents and tools)
- ✅ Completed Notebook 03: LlamaStack Core Features (understanding of Chat and RAG)
- ✅ LlamaStack server running (see Module README)
- ✅ MongoDB MCP server running (or access to one)
- ✅ Python environment with dependencies installed (`llama-stack-client`, `httpx`)

**Important:** This notebook requires an MCP server to be running. We'll use a MongoDB MCP server as an example, but the concepts apply to any MCP server.

---

## 📋 Step-by-Step Guide

### Step 1: Setup and Configuration

**What we're doing:** Setting up the environment and connecting to LlamaStack.

**Why:** We need to establish connections before we can use MCP tools with agents.

**What to expect:**
- Import required libraries
- Load configuration from shared config system
- Connect to LlamaStack server
- Verify everything is working


In [ ]:
# Import required libraries
import sys
from pathlib import Path
import os
import httpx
import time
import json

# Add src directory to path for shared configuration
root_dir = Path("../..").resolve()
sys.path.insert(0, str(root_dir / "src"))

# Import centralized configuration
from config import LLAMA_STACK_URL, MODEL, CONFIG, MCP_MONGODB_URL

# Import LlamaStack client
from llama_stack_client import LlamaStackClient

print("✅ Libraries imported successfully!")
print(f"📡 LlamaStack URL: {LLAMA_STACK_URL}")
print(f"🤖 Model: {MODEL}")
print(f"🔌 MCP MongoDB URL: {MCP_MONGODB_URL if MCP_MONGODB_URL else 'Not configured'}")

# Verify configuration
if not LLAMA_STACK_URL:
    raise ValueError(
        "LLAMA_STACK_URL is not configured!\n"
        "Please run: ./scripts/setup-env.sh"
    )

# Initialize LlamaStack client
client = LlamaStackClient(base_url=LLAMA_STACK_URL)

# Verify connection
try:
    models = client.models.list()
    model_count = len(models.data) if hasattr(models, 'data') else len(models)
    print(f"\n✅ Connected to LlamaStack")
    print(f"   Available models: {model_count}")
except Exception as e:
    print(f"\n❌ Cannot connect to LlamaStack: {e}")
    raise


**What happened:** After running the code, you should see successful connections to LlamaStack. The configuration is loaded from the shared `src/config.py` system, which auto-detects your environment.

**Key takeaway:** The shared configuration system makes it easy to switch between environments (local, OpenShift, etc.) without changing code.


### Step 2: Understanding MCP Server Connection

**What we're doing:** Learning how agents connect to MCP servers.

**Why:** Before agents can use MCP tools, they need to be connected to an MCP server. This is like connecting to a database or API - you need the connection details first.

**What to expect:**
- We'll check if an MCP server URL is configured
- Understand how MCP servers are registered with agents
- See how agents discover available tools from MCP servers

**Important:** For this notebook, we'll use a MongoDB MCP server as an example. The concepts apply to any MCP server.


In [ ]:
# Check MCP server configuration
if MCP_MONGODB_URL:
    print(f"✅ MCP MongoDB URL configured: {MCP_MONGODB_URL}")
    print("\n💡 This notebook will use the MongoDB MCP server.")
    print("   The MCP server exposes database operations as tools that agents can use.")
else:
    print("⚠️  MCP_MONGODB_URL not configured")
    print("\n💡 To use MCP tools, you need:")
    print("   1. An MCP server running (e.g., MongoDB MCP server)")
    print("   2. The MCP server URL configured in .env or environment variables")
    print("\n   For this notebook, we'll demonstrate the concepts.")
    print("   In production, you would configure the MCP server URL.")


**What happened:** After running the code, you can see whether an MCP server URL is configured. If configured, the agent will be able to connect to it. If not, you'll see instructions on how to set it up.

**Key takeaway:** MCP servers need to be configured before agents can use them. The configuration tells the agent where to find the MCP server and how to connect to it.


### Step 3: Creating an Agent with MCP Tools

**What we're doing:** Creating an agent and connecting it to MCP tools.

**Why:** Agents need to be configured with MCP server connections to use MCP tools. This is similar to how you configure database connections in applications.

**What to expect:**
- Create an agent using LlamaStack
- Register MCP server with the agent
- See how the agent discovers available MCP tools
- Understand the difference between client-side tools and MCP tools


In [ ]:
# Helper function to convert objects to dictionaries
def to_dict(obj):
    """Convert object to dictionary"""
    if isinstance(obj, dict):
        return obj
    if hasattr(obj, 'model_dump'):
        return obj.model_dump()
    if hasattr(obj, 'dict'):
        return obj.dict()
    if hasattr(obj, '__dict__'):
        return {k: v for k, v in obj.__dict__.items() if not k.startswith('_')}
    return {}

# Create an agent
# Note: MCP server registration happens when creating the agent
# For this example, we'll create a basic agent
# In production, you would register MCP servers during agent creation

print("=" * 60)
print("Creating Agent")
print("=" * 60)

try:
    # Create agent with model
    agent = client.alpha.agents.create(
        name="MCP Tools Demo Agent",
        model=MODEL,
        description="Agent that demonstrates MCP tool usage"
    )
    print(f"✅ Agent created: {agent.agent_id}")
    print(f"   Name: {agent.name}")
    print(f"   Model: {agent.model}")
    
    # Create a session for the agent
    session = client.alpha.agents.sessions.create(
        agent_id=agent.agent_id
    )
    print(f"\n✅ Session created: {session.session_id}")
    
except Exception as e:
    print(f"\n❌ Error creating agent: {e}")
    import traceback
    traceback.print_exc()
    raise

print("\n💡 Note: MCP server registration would happen here in production.")
print("   The agent would be configured with MCP server URLs during creation.")


**What happened:** After creating the agent, it's ready to use MCP tools. The agent can discover available tools from any configured MCP servers and use them when needed.

**Key takeaway:** Agents discover MCP tools dynamically - they don't need to be pre-configured with specific tool names. The MCP server tells the agent what tools are available, and the agent decides which ones to use based on the task.


### Step 4: Querying the Agent with MCP Tools

**What we're doing:** Querying an agent that uses MCP tools to interact with a database.

**Why:** This demonstrates how agents use MCP tools in practice - the agent receives a query, decides it needs database information, calls the MCP tool, and uses the results to answer.

**What to expect:**
- Send a query to the agent
- Watch the agent use MCP tools to query the database
- See how tool results are integrated into the agent's response
- Understand the complete flow: query → tool call → results → response


In [ ]:
# Query the agent
print("=" * 60)
print("Querying Agent with MCP Tools")
print("=" * 60)

query = "What collections are in the mcp_demo database?"
print(f"\n👤 Query: {query}\n")

# Create turn with streaming
try:
    turn_stream = client.alpha.agents.turn.create(
        agent_id=agent.agent_id,
        session_id=session.session_id,
        messages=[{"role": "user", "content": query}],
        stream=True
    )
    print("✅ Turn created, streaming response...")
except Exception as e:
    print(f"\n❌ Error creating turn: {e}")
    raise

# Extract turn_id from streaming response
print("\n📋 Extracting turn_id from stream...")
turn_id = None
for chunk in turn_stream:
    if hasattr(chunk, 'event') and chunk.event and hasattr(chunk.event, 'payload'):
        payload = chunk.event.payload
        d = to_dict(payload)
        turn_id = d.get('turn_id')
        if not turn_id and 'step_details' in d:
            step_details = to_dict(d['step_details'])
            turn_id = step_details.get('turn_id')
        if turn_id:
            break

if turn_id:
    print(f"✅ Turn ID: {turn_id}")
else:
    print("⚠️  Could not extract turn_id from stream")


**What happened:** After querying the agent, it processes your request and decides to use MCP tools to query the database. The agent calls the appropriate MCP tool, receives results, and formulates a response.

**Key takeaway:** MCP tools enable agents to interact with external systems seamlessly. The agent doesn't need to know database connection details - it just calls the MCP tool and gets results back.


In [ ]:
# Retrieve turn results
if turn_id:
    print("=" * 60)
    print("Retrieving Turn Results")
    print("=" * 60)
    
    print("\n⏳ Waiting for turn to complete...")
    time.sleep(2)  # Give the turn time to complete
    
    try:
        # Retrieve the turn
        response = httpx.get(
            f"{LLAMA_STACK_URL}/v1alpha/agents/{agent.agent_id}/session/{session.session_id}/turn/{turn_id}",
            verify=False,
            timeout=30
        )
        response.raise_for_status()
        
        # Handle null response
        if not response.text or response.text.strip() == 'null':
            print("   ⚠️  Turn not ready yet (response was null)")
            print("   💡 The turn may still be processing. Try waiting a bit longer.")
            data = None
        else:
            data = response.json()
        
        if data is None:
            print("   ⚠️  Could not retrieve turn data")
            print("   💡 This might mean the turn is still processing or the turn_id is incorrect")
        else:
            print("✅ Turn data retrieved successfully!")
            
            # Extract messages and steps
            messages = data.get('messages', [])
            steps = data.get('steps', [])
            
            print(f"\n📊 Turn Summary:")
            print(f"   Messages: {len(messages)}")
            print(f"   Steps: {len(steps)}")
            
    except Exception as e:
        print(f"\n❌ Error retrieving turn: {e}")
        import traceback
        traceback.print_exc()
        data = None
else:
    print("\n⚠️  Cannot retrieve turn results without turn_id")
    data = None


**What happened:** After retrieving the turn results, you can see the complete interaction - the agent's reasoning steps, tool calls, and final response.

**Key takeaway:** The turn results show you exactly how the agent used MCP tools - which tools were called, what arguments were passed, and what results were returned. This transparency helps you understand and debug agent behavior.


### Step 5: Analyzing Tool Usage (Optional)

**What we're doing:** Analyzing how the agent used MCP tools during the interaction.

**Why:** Understanding tool usage helps you see exactly what the agent did and how it used external services.

**What to expect:**
- See which MCP tools were called
- View the arguments passed to each tool
- See the results returned from tools
- Understand the complete tool execution flow


In [ ]:
# Analyze tool usage from steps (tool-specific only)
import json

if data:
    print("=" * 60)
    print("Analyzing Tool Usage")
    print("=" * 60)
    
    steps = data.get('steps', [])
    
    if steps:
        # Filter and show only tool-related steps
        tool_steps = []
        for i, step in enumerate(steps):
            step_type = step.get('type', '') or step.get('step_type', '')
            
            # Check for tool calls in inference steps
            if step_type == 'inference':
                model_response = step.get('model_response', {})
                tool_calls = model_response.get('tool_calls', [])
                if tool_calls:
                    tool_steps.append((i, step, 'tool_call', tool_calls))
            
            # Check for tool execution steps
            if step_type == 'tool_execution':
                tool_execution = step.get('tool_execution', {})
                if tool_execution:
                    tool_steps.append((i, step, 'tool_execution', tool_execution))
            
            # Check for tool result steps
            if step_type == 'tool_result':
                tool_result = step.get('tool_result', {})
                if tool_result:
                    tool_steps.append((i, step, 'tool_result', tool_result))
        
        if tool_steps:
            print(f"\n🔧 Found {len(tool_steps)} tool-related step(s):\n")
            
            for step_idx, step, tool_type, tool_data in tool_steps:
                print(f"{'='*60}")
                print(f"Step {step_idx}: {tool_type}")
                print(f"{'='*60}")
                
                if tool_type == 'tool_call':
                    print("\n📋 Tool Calls from Inference Step:")
                    for tool_call in tool_data:
                        call_id = tool_call.get('call_id', 'unknown')
                        tool_name = tool_call.get('tool_name', 'unknown')
                        arguments = tool_call.get('arguments', '{}')
                        print(f"\n  🔧 Tool: {tool_name}")
                        print(f"     Call ID: {call_id}")
                        try:
                            args_dict = json.loads(arguments) if isinstance(arguments, str) else arguments
                            print(f"     Arguments: {json.dumps(args_dict, indent=6)}")
                        except:
                            print(f"     Arguments: {arguments}")
                
                elif tool_type == 'tool_execution':
                    print("\n⚙️  Tool Execution:")
                    tool_name = tool_data.get('tool_name', 'unknown')
                    call_id = tool_data.get('call_id', 'unknown')
                    print(f"  Tool: {tool_name}")
                    print(f"  Call ID: {call_id}")
                    if 'arguments' in tool_data:
                        args = tool_data.get('arguments', {})
                        print(f"  Arguments: {json.dumps(args, indent=4, default=str)}")
                
                elif tool_type == 'tool_result':
                    print("\n📤 Tool Result:")
                    tool_name = tool_data.get('name', tool_data.get('tool_name', 'unknown'))
                    result_content = tool_data.get('content', '')
                    print(f"  Tool: {tool_name}")
                    if result_content:
                        result_str = str(result_content)
                        if len(result_str) > 500:
                            print(f"  Result: {result_str[:500]}...")
                            print(f"  (Full result: {len(result_str)} characters)")
                        else:
                            print(f"  Result: {result_str}")
                
                print()
        else:
            print("\n⚠️  No tool-related steps found")
            print("   💡 The agent may not have used any tools, or tool information is in a different format")
            print("\n   Available step types:")
            for i, step in enumerate(steps):
                step_type = step.get('type', '') or step.get('step_type', '')
                print(f"     Step {i}: {step_type}")
    else:
        print("\n⚠️  No steps found in turn data")
else:
    print("\n⚠️  No turn data available")
    print("   💡 Make sure Part 5 completed successfully")


**What happened:** The analysis shows you exactly which MCP tools were used, what data was passed to them, and what results they returned. This gives you full visibility into how the agent interacted with external services.

**Key takeaway:** Tool usage analysis is crucial for debugging and understanding agent behavior. It shows you the complete flow from query to tool call to results to final response.


### Step 6: Extracting the Agent's Response (Optional)

**What we're doing:** Extracting and displaying the agent's final response.

**Why:** The agent's response is the final output that combines all the information gathered from MCP tools.

**What to expect:**
- See the agent's complete response
- Understand how tool results were integrated into the response
- See which tools were used in generating the response


In [ ]:
# Extract the agent's response
if data:
    print("=" * 60)
    print("Extracting Agent Response")
    print("=" * 60)
    
    messages = data.get('messages', [])
    steps = data.get('steps', [])
    response_text = None
    
    # Try messages first
    print("\n🔍 Checking messages...")
    for msg in reversed(messages):
        if msg.get('role') == 'assistant':
            content = msg.get('content', '')
            if isinstance(content, str) and content.strip():
                response_text = content
                print("   ✅ Found response in messages")
                break
            elif isinstance(content, list) and content:
                for item in content:
                    if isinstance(item, dict):
                        text = item.get('text', '')
                        if text:
                            response_text = text
                            print("   ✅ Found response in messages (list format)")
                            break
                    elif isinstance(item, str) and item.strip():
                        response_text = item
                        print("   ✅ Found response in messages (list format)")
                        break
                if response_text:
                    break
    
    # If not found in messages, check steps
    if not response_text:
        print("   ⚠️  Not found in messages, checking steps...")
        for step in reversed(steps):
            # Try inference step
            if step.get('type') == 'inference' or step.get('step_type') == 'inference':
                model_response = step.get('model_response', {})
                content = model_response.get('content', '')
                if isinstance(content, str) and content.strip():
                    response_text = content
                    print("   ✅ Found response in inference step")
                    break
            
            # Try any step with model_response
            if 'model_response' in step:
                model_response = step.get('model_response', {})
                content = model_response.get('content', '')
                if isinstance(content, str) and content.strip():
                    response_text = content
                    print("   ✅ Found response in step model_response")
                    break
    
    # Display the response
    print("\n" + "=" * 60)
    print("Agent Response")
    print("=" * 60)
    
    if response_text:
        print(f"\n💬 {response_text}\n")
        
        # Show tools used
        tools_used = []
        for step in steps:
            if step.get('type') == 'tool_call' or step.get('step_type') == 'tool_call':
                tool_call = step.get('tool_call', {})
                tool_name = tool_call.get('name', 'unknown')
                if tool_name not in tools_used:
                    tools_used.append(tool_name)
        
        if tools_used:
            print(f"🔧 Tools used: {', '.join(tools_used)}\n")
        else:
            print("💡 No tools were used in this response\n")
    else:
        print("\n⚠️  Could not extract response text")
        print("   💡 The turn may still be processing, or response format is unexpected")
        print("   💡 Check the steps above for more details")


**What happened:** The agent's response combines information from MCP tools with its reasoning to provide a complete answer. You can see which tools were used and how their results influenced the final response.

**Key takeaway:** MCP tools provide data, but the agent's reasoning combines that data into a coherent, useful response. This is the power of combining external tools with AI reasoning.


In [ ]:
# Test with multiple queries
import json

def extract_tools_from_steps(steps):
    """Extract tool names from steps (checking both tool_execution and inference steps)"""
    tools_used = []
    for step in steps:
        step_type = step.get('type', '') or step.get('step_type', '')
        
        # Check tool_execution steps
        if step_type == 'tool_execution':
            tool_execution = step.get('tool_execution', {})
            tool_name = tool_execution.get('tool_name', '')
            if tool_name and tool_name not in tools_used:
                tools_used.append(tool_name)
        
        # Check inference steps for tool_calls
        elif step_type == 'inference':
            model_response = step.get('model_response', {})
            tool_calls = model_response.get('tool_calls', [])
            for tool_call in tool_calls:
                tool_name = tool_call.get('tool_name', '')
                if tool_name and tool_name not in tools_used:
                    tools_used.append(tool_name)
    
    return tools_used

def extract_response_text(data):
    """Extract response text from messages or steps"""
    messages = data.get('messages', [])
    steps = data.get('steps', [])
    response_text = None
    
    # Try messages first
    for msg in reversed(messages):
        if msg.get('role') == 'assistant':
            content = msg.get('content', '')
            if isinstance(content, str) and content.strip():
                response_text = content
                break
            elif isinstance(content, list) and content:
                for item in content:
                    if isinstance(item, dict):
                        text = item.get('text', '')
                        if text:
                            response_text = text
                            break
                    elif isinstance(item, str) and item.strip():
                        response_text = item
                        break
                if response_text:
                    break
    
    # If not found in messages, check steps
    if not response_text:
        for step in reversed(steps):
            if step.get('type') == 'inference' or step.get('step_type') == 'inference':
                model_response = step.get('model_response', {})
                content = model_response.get('content', '')
                if isinstance(content, str) and content.strip():
                    response_text = content
                    break
    
    return response_text

def retrieve_turn_with_retry(llamastack_url, agent_id, session_id, turn_id, max_retries=5, initial_wait=2):
    """Retrieve turn with retry logic"""
    for attempt in range(max_retries):
        wait_time = initial_wait * (2 ** attempt)  # Exponential backoff
        if attempt > 0:
            print(f"   ⏳ Retry {attempt}/{max_retries-1} (waiting {wait_time}s)...")
            time.sleep(wait_time)
        else:
            time.sleep(wait_time)
        
        try:
            response = httpx.get(
                f"{llamastack_url}/v1alpha/agents/{agent_id}/session/{session_id}/turn/{turn_id}",
                verify=False,
                timeout=30
            )
            response.raise_for_status()
            
            if response.text and response.text.strip() != 'null':
                data = response.json()
                steps = data.get('steps', [])
                
                # Check if turn is complete (has final inference step with content)
                is_complete = False
                for step in reversed(steps):
                    if step.get('type') == 'inference' or step.get('step_type') == 'inference':
                        model_response = step.get('model_response', {})
                        content = model_response.get('content', '')
                        if content and content.strip():
                            is_complete = True
                            break
                
                if is_complete or len(steps) > 0:
                    return data
                elif attempt < max_retries - 1:
                    continue  # Try again
            
        except Exception as e:
            if attempt < max_retries - 1:
                continue
            else:
                raise
    
    return None

print("=" * 60)
print("Testing Multiple Queries")
print("=" * 60)

test_queries = [
    "What collections are in the mcp_demo database?",
    "How many documents are in the incidents collection?",
    "Show me the first document from the incidents collection",
]

for i, query in enumerate(test_queries, 1):
    print(f"\n{'='*60}")
    print(f"Test {i}: {query}")
    print('='*60)
    
    try:
        # Create turn
        print("   📤 Creating turn...")
        turn_stream = client.alpha.agents.turn.create(
            agent_id=agent.agent_id,
            session_id=session.session_id,
            messages=[{"role": "user", "content": query}],
            stream=True
        )
        
        # Get turn_id
        print("   🔍 Extracting turn_id...")
        turn_id = None
        for chunk in turn_stream:
            if hasattr(chunk, 'event') and chunk.event and hasattr(chunk.event, 'payload'):
                payload = chunk.event.payload
                d = to_dict(payload)
                turn_id = d.get('turn_id')
                if not turn_id and 'step_details' in d:
                    step_details = to_dict(d['step_details'])
                    turn_id = step_details.get('turn_id')
                if turn_id:
                    break
        
        if turn_id:
            print(f"   ✅ Turn ID: {turn_id}")
            print("   ⏳ Retrieving turn results...")
            
            # Retrieve with retry
            data = retrieve_turn_with_retry(
                llamastack_url, 
                agent.agent_id, 
                session.session_id, 
                turn_id
            )
            
            if data:
                steps = data.get('steps', [])
                messages = data.get('messages', [])
                
                # Extract tools used
                tools_used = extract_tools_from_steps(steps)
                
                # Extract response
                response_text = extract_response_text(data)
                
                # Display results
                print(f"\n   📊 Results:")
                print(f"      Steps: {len(steps)}")
                print(f"      Messages: {len(messages)}")
                
                if tools_used:
                    print(f"      🔧 Tools used: {', '.join(tools_used)}")
                else:
                    print(f"      🔧 Tools used: None")
                    # Show step types for debugging
                    step_types = [step.get('type', '') or step.get('step_type', '') for step in steps]
                    if step_types:
                        print(f"      📋 Step types: {', '.join(step_types)}")
                
                if response_text:
                    # Truncate long responses
                    display_text = response_text[:300] + "..." if len(response_text) > 300 else response_text
                    print(f"      💬 Response: {display_text}")
                else:
                    print(f"      ⚠️  No response text found")
                    # Try to show what we have
                    if steps:
                        print(f"      💡 Check Part 5 and Part 6 for detailed step information")
            else:
                print(f"   ⚠️  Could not retrieve turn data after retries")
        else:
            print("   ⚠️  Could not get turn_id from stream")
            
    except Exception as e:
        print(f"   ❌ Error: {e}")
        import traceback
        traceback.print_exc()

print("\n✅ Multiple query test complete!")
print("\n💡 Tip: For detailed tool usage and step information, check Part 5 and Part 6 above.")


---

## 🎓 Key Takeaways

**What we learned:**

1. **MCP (Model Context Protocol)** is a standardized way to connect agents to external tools and services
2. **MCP vs. Client-Side Tools:** MCP connects to external services, client-side tools run in your process
3. **MCP Servers** expose tools via a standard protocol - databases, APIs, monitoring systems, anything!
4. **Dynamic Tool Discovery:** Agents discover MCP tools automatically - no need to pre-configure them
5. **Production-Ready:** MCP enables agents to interact with real IT infrastructure safely and securely

**The big picture:**
- MCP provides a standardized way to connect agents to any external service
- This enables production agents that interact with databases, APIs, monitoring systems, and more
- MCP tools run on dedicated servers, making them scalable and secure
- The protocol is standardized, so agents can work with any MCP-compatible service

**For IT operations:**
- Connect agents to your monitoring systems via MCP
- Integrate with ticketing systems, databases, and APIs
- Build production-ready agents that interact with your infrastructure
- Use standardized protocol for easy integration and maintenance

**When to use MCP vs. Client-Side Tools:**
- **Use MCP:** External services, production systems, shared tools, databases, APIs
- **Use Client-Side:** Simple operations, prototyping, no external dependencies

---

## 🔗 Next Steps

**Ready for more?** In **Notebook 05**, we'll explore:
- **Safety Shields** - Content moderation and safety checks for agents
- **How to protect agents** from harmful inputs and outputs
- **Implementing safeguards** for production deployments

**The fun part:** You'll learn how to make agents safe and secure for production use!

**Next notebook:** `05_safety_shields.ipynb` - Safety Shields and Content Moderation

**Related concepts:**
- MCP Protocol documentation
- Building custom MCP servers
- Agent security best practices (covered in Notebook 05)
